In [ ]:
"""
Created on Tue Mar 16 10:57 2021

reformat basic mask to give 0-1-2 and floats 
to be used after "prepare_data_NEMO.sh"

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from cdo import Cdo
import time

In [ ]:
%matplotlib qt5

In [ ]:
cdo = Cdo()
print('this is CDO version %s'%(cdo.version()))

In [ ]:
nemo_run = 'EPM026' # 'EPM026','EPM031', 'EPM034'
if nemo_run in ['EPM026','EPM031']:
    ystart = 2049
    yend = 2058
elif nemo_run == 'EPM034':
    ystart = 2119
    yend = 2128   

In [ ]:
inputpath_data='/bettik/burgardc/DATA/NN_PARAM/interim/NEMO_eORCA025.L121_'+nemo_run+'_ANT_STEREO/'

PREPARE THE FLOAT MASK (0 = ocean, 1 = ice shelf, 2 = ground)

In [ ]:
for yy in range(ystart,yend+1):
    mask_file = xr.open_dataset(inputpath_data+'lsmask_0-1-2_Ant_'+str(yy)+'.nc')
    file_map_orig = xr.open_dataset(inputpath_data+'mask_variables_of_interest_Ant_'+str(yy)+'.nc')
    ls_mask012 = mask_file['tmask'].squeeze().astype('float')
    ls_mask012.to_dataset(name='ls_mask012').to_netcdf(inputpath_data+'custom_lsmask_Ant_'+str(yy)+'.nc')

SEPARATE OTHER VARIABLES

In [ ]:
for yy in range(ystart,yend+1):
    print(yy)
    file_map_orig = xr.open_dataset(inputpath_data+'mask_variables_of_interest_Ant_'+str(yy)+'.nc')
    isfdraft = file_map_orig['isf_draft'].squeeze().drop('time')
    isfdraft.to_dataset().to_netcdf(inputpath_data+'isfdraft_Ant_'+str(yy)+'.nc')

    T_levs = file_map_orig['e3w_0'].squeeze()
    T_levs_cumsum = T_levs.cumsum(dim='nav_lev') - (T_levs.isel(nav_lev=0)/2)
    gdept_0 = T_levs.copy(data=T_levs_cumsum).rename('gdept_0')
    
    bathy_metry = file_map_orig[['bathy_metry']].squeeze().drop('time')
    other_mask_vars = xr.merge([gdept_0,bathy_metry])
    other_mask_vars.to_netcdf(inputpath_data+'other_mask_vars_Ant_'+str(yy)+'.nc')

INTERPOLATE TO STEREOGRAPHIC GRID

In [ ]:
for yy in range(ystart,yend+1):
    print('interpolate float mask '+str(yy))
    time_start = time.time()
    cdo.remapbil(inputpath_data+'stereo_grid.nc', input = inputpath_data+'custom_lsmask_Ant_'+str(yy)+'.nc', output = inputpath_data+'custom_lsmask_Ant_stereo_'+str(yy)+'.nc')
    timelength = time.time() - time_start
    print(timelength)

In [ ]:
for yy in range(ystart,yend+1):
    print('interpolate isfdraft '+str(yy))
    time_start = time.time()
    cdo.remapbil(inputpath_data+'stereo_grid.nc', input = inputpath_data+'isfdraft_Ant_'+str(yy)+'.nc', output = inputpath_data+'isfdraft_Ant_stereo_'+str(yy)+'.nc')
    timelength = time.time() - time_start
    print(timelength)

Other variables of interest

In [ ]:
for yy in range(ystart,yend+1):
    print('computing mask '+str(yy))
    time_start = time.time()
    cdo.remapbil(inputpath_data+'stereo_grid.nc', input = inputpath_data+'other_mask_vars_Ant_'+str(yy)+'.nc', output = inputpath_data+'other_mask_vars_Ant_stereo_'+str(yy)+'.nc')
    timelength = time.time() - time_start
    print(timelength)

In [ ]:
for yy in range(ystart,yend+1):
    print('creating mean depth coordinate '+str(yy))
    time_start = time.time()
    cdo.fldmean(input=' -selvar,gdept_0 '+inputpath_data+'other_mask_vars_Ant_stereo_'+str(yy)+'.nc', 
              output = inputpath_data+'mask_depth_coord_Ant_stereo_'+str(yy)+'.nc') 
    timelength = time.time() - time_start
    print(timelength)

PREPARE AN ICE SHELF CONCENTRATION

In [ ]:
for yy in range(ystart,yend+1):
    print('prepare an ice shelf concentration '+str(yy))
    float_ls_mask = xr.open_dataset(inputpath_data+'custom_lsmask_Ant_stereo_'+str(yy)+'.nc')

    lsmask_012_withcenter = float_ls_mask['ls_mask012'].where(np.isfinite(float_ls_mask['ls_mask012']),2)
    should_not_be_ground = (float_ls_mask['ls_mask012']==0) | (isfdraft['isfdraft']>0)
    lsmask_below2 = float_ls_mask['ls_mask012'].where(should_not_be_ground,2)

    lsmask_below2.to_dataset(name='ls_mask012').to_netcdf(inputpath_data+'custom_lsmask_Ant_stereo_clean_'+str(yy)+'.nc')

    first_isfdraft_mask = float_ls_mask['ls_mask012'].where(isfdraft['isfdraft']>0,0) # ignore the points of transition between ocean and land
    isfdraft_conc = first_isfdraft_mask.where(first_isfdraft_mask <= 1,(2 - first_isfdraft_mask))
    isfdraft_conc.to_dataset(name='isfdraft_conc').to_netcdf(inputpath_data+'isfdraft_conc_Ant_stereo_'+str(yy)+'.nc')

    corrected_isfdraft_raw = isfdraft['isfdraft']/isfdraft_conc
    corrected_isfdraft = corrected_isfdraft_raw.where(corrected_isfdraft_raw<=3000,3000).where(corrected_isfdraft_raw>0) # cut unrealistically high drafts
    corrections = corrected_isfdraft.to_dataset(name='corrected_isfdraft')

    mask_vars = xr.open_dataset(inputpath_data+'other_mask_vars_Ant_stereo_'+str(yy)+'.nc')
    corrected_bathy_raw = mask_vars['bathy_metry']/isfdraft_conc # not realistically cuttable because bathymetry is set to 0 on land...
    corrected_bathy = corrected_bathy_raw.where(corrected_bathy_raw<=3000,3000).where(corrected_bathy_raw>0)

    corrections['corrected_isf_bathy'] = corrected_bathy

    corrections.to_netcdf(inputpath_data+'corrected_draft_bathy_isf_'+str(yy)+'.nc')

FINISHED? THEN MOVE TO "regridding_vars_cdo.ipynb"